<a href="https://colab.research.google.com/github/KalharaBatangala/RGB-Thermal-Feature-Fusion/blob/main/RGB_Thermal_Feature_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import xml.etree.ElementTree as ET

xml_path = "/content/drive/MyDrive/Research/RGB-termal feature fusion project/LLVIP/Annotations/010001.xml"
tree = ET.parse(xml_path)
root = tree.getroot()

for obj in root.findall('object'):
    print(obj.find('name').text)
    bbox = obj.find('bndbox')
    print(
        bbox.find('xmin').text,
        bbox.find('ymin').text,
        bbox.find('xmax').text,
        bbox.find('ymax').text
    )


person
287 428 351 662
person
351 391 424 642
person
466 367 550 614
person
700 354 761 585
person
704 517 806 794
person
1124 22 1196 245


In [ ]:
from PIL import Image

img = Image.open("/content/drive/MyDrive/Research/RGB-termal feature fusion project/LLVIP/visible/train/010001.jpg")
print(img.size)


(1280, 1024)


In [ ]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

# Paths
BASE_PATH = "/content/drive/MyDrive/Research/RGB-termal feature fusion project/LLVIP"
ANNOTATIONS_PATH = os.path.join(BASE_PATH, "Annotations")
IMAGES_PATH = os.path.join(BASE_PATH, "visible/train")

OUTPUT_LABELS = "/content/drive/MyDrive/Research/LLVIP_YOLO_RGB/labels/train"
OUTPUT_IMAGES = "/content/drive/MyDrive/Research/LLVIP_YOLO_RGB/images/train"

os.makedirs(OUTPUT_LABELS, exist_ok=True)
os.makedirs(OUTPUT_IMAGES, exist_ok=True)

for xml_file in os.listdir(ANNOTATIONS_PATH):
    if not xml_file.endswith(".xml"):
        continue

    xml_path = os.path.join(ANNOTATIONS_PATH, xml_file)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(IMAGES_PATH, image_name)

    if not os.path.exists(image_path):
        continue

    img = Image.open(image_path)
    img_w, img_h = img.size

    label_lines = []

    for obj in root.findall("object"):
        class_name = obj.find("name").text.strip()
        if class_name != "person":
            continue

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)

        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        label_lines.append(f"0 {x_center} {y_center} {width} {height}")

    if label_lines:
        with open(os.path.join(OUTPUT_LABELS, xml_file.replace(".xml", ".txt")), "w") as f:
            f.write("\n".join(label_lines))

    # Copy image
    os.system(f"cp '{image_path}' '{OUTPUT_IMAGES}'")


**Create validation split**

In [ ]:
import random
import shutil

IMG_DIR = "/content/drive/MyDrive/Research/LLVIP_YOLO_RGB/images/train"
LBL_DIR = "/content/drive/MyDrive/Research/LLVIP_YOLO_RGB/labels/train"

VAL_IMG = "/content/drive/MyDrive/Research/LLVIP_YOLO_RGB/images/val"
VAL_LBL = "/content/drive/MyDrive/Research/LLVIP_YOLO_RGB/labels/val"

os.makedirs(VAL_IMG, exist_ok=True)
os.makedirs(VAL_LBL, exist_ok=True)

images = os.listdir(IMG_DIR)
random.shuffle(images)

val_count = int(0.2 * len(images))
val_images = images[:val_count]

for img in val_images:
    shutil.move(os.path.join(IMG_DIR, img), os.path.join(VAL_IMG, img))
    lbl = img.replace(".jpg", ".txt")
    shutil.move(os.path.join(LBL_DIR, lbl), os.path.join(VAL_LBL, lbl))


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.7 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # nano model (good for baseline)

model.train(
    data="/content/drive/MyDrive/Research/RGB-termal feature fusion project/llvip_rgb.yaml",
    epochs=10,
    imgsz=640,
    batch=8,
    name="llvip_rgb_baseline"
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.242 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Research/RGB-termal feature fusion project/llvip_rgb.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7,

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b27204c0230>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 